## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

see https://classroom.udacity.com/nanodegrees/nd013/parts/edf28735-efc1-4b99-8fbb-ba9c432239c8/modules/5d1efbaa-27d0-4ad5-a67a-48729ccebd9c/lessons/626f183c-593e-41d7-a828-eda3c6122573/concepts/e6e02d4d-7c80-4bed-a79f-869ef496831b

In [ ]:
def get_width(image):
    return image.shape[1]

def get_height(image):
    return image.shape[0]

def get_imageSize_for_cv2(image):
    return (get_width(image), get_height(image))

class Pix2Meter:    
    def __init__(self, ym_per_pix, xm_per_pix):
        # meters per pixel in y dimension
        self.ym_per_pix = ym_per_pix
        # meters per pixel in x dimension
        self.xm_per_pix = xm_per_pix
    
    def y_pix_2_meter(self, pix):
        return pix * self.ym_per_pix

    def x_pix_2_meter(self, pix):
        return pix * self.xm_per_pix

def create_default_pix2Meter():
    return Pix2Meter(ym_per_pix = 30/720, xm_per_pix = 3.7/700)

def savefig(image):
    plt.savefig(image, bbox_inches = 'tight')

## Compute the camera calibration matrix and distortion coefficients given a set of chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

def get_objpoints_imgpoints(draw):
    patternSize = (9, 6)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((np.prod(patternSize), 3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, patternSize, None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            if draw:
                img = cv2.drawChessboardCorners(img, patternSize, corners, ret)
                cv2.imshow('img', img)
                cv2.waitKey(500)

    cv2.destroyAllWindows()
    return objpoints, imgpoints

In [ ]:
import pickle
import os

def calibrateCamera():
    objpoints, imgpoints = get_objpoints_imgpoints(draw = False)
    img = cv2.imread('camera_cal/calibration1.jpg')
    # Do camera calibration given object points and image points
    _, cameraMatrix, distCoeffs, _, _ = cv2.calibrateCamera(objpoints,
                                                            imgpoints,
                                                            get_imageSize_for_cv2(img),
                                                            None,
                                                            None)
    calibration_data = {"cameraMatrix": cameraMatrix, "distCoeffs": distCoeffs}
    return calibration_data

def get_calibrateCamera_pickle_file():
    return 'camera_cal/wide_dist_pickle.p'

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
def save_calibration_data(calibration_data):
    pickle.dump(calibration_data, open(get_calibrateCamera_pickle_file(), "wb"))

# Read in the saved camera matrix and distortion coefficients
def load_calibration_data():
    calibration_data = pickle.load(open(get_calibrateCamera_pickle_file(), "rb"))
    return calibration_data

def get_calibration_data():
    if not os.path.isfile(get_calibrateCamera_pickle_file()):
        save_calibration_data(calibrateCamera())
    return load_calibration_data()

## Apply a distortion correction to raw images

adapted from https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

In [ ]:
def undistort(image, calibration_data):
    return cv2.undistort(image,
                         calibration_data['cameraMatrix'],
                         calibration_data['distCoeffs'],
                         None,
                         calibration_data['cameraMatrix'])

In [ ]:
%matplotlib inline

def visualize_undistortion(original_image, undistorted_image):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    ax1.imshow(original_image)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(undistorted_image)
    ax2.set_title('Undistorted Image', fontsize=30)

In [ ]:
original_image = cv2.imread('camera_cal/calibration1.jpg')
undistorted_image = undistort(original_image, get_calibration_data())
# cv2.imwrite('output_images/calibration1_undistorted.png', dst)

visualize_undistortion(original_image, undistorted_image)
savefig('output_images/calibration1_undistorted.png')

In [ ]:
original_image = cv2.cvtColor(cv2.imread('test_images/test1.jpg'), cv2.COLOR_BGR2RGB)
undistorted_image = undistort(original_image, get_calibration_data())

visualize_undistortion(original_image, undistorted_image)
savefig('output_images/test1_undistorted.png')

## Use color transforms, gradients, etc., to create a thresholded binary image

see https://classroom.udacity.com/nanodegrees/nd013/parts/edf28735-efc1-4b99-8fbb-ba9c432239c8/modules/5d1efbaa-27d0-4ad5-a67a-48729ccebd9c/lessons/144d538f-335d-454d-beb2-b1736ec204cb/concepts/a1b70df9-638b-46bb-8af0-12c43dcfd0b4

Some interesting things to explore might include: the H channel, different threshold values for color and gradient binary images, and even a different color space, like HSV!

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def apply_thresholds(image, thresholds):
    result = np.zeros_like(image)
    result[(image >= thresholds[0]) & (image <= thresholds[1])] = 1
    return result

    
def detect_edges_using_x_derivative(image, sx_thresh):
    # Sobel x
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    return apply_thresholds(scaled_sobel, sx_thresh)


def combine_with_or(binary_img1, binary_img2):
    combined_binary = np.zeros_like(binary_img1)
    combined_binary[(binary_img1 == 1) | (binary_img2 == 1)] = 1
    return combined_binary


def create_color_binary_image(green_image, blue_image):
    red_image = np.zeros_like(green_image)
    return np.dstack((red_image, green_image, blue_image)) * 255
    

def get_l_channel_and_s_channel(image):
    # Convert to HLS color space and separate the S and L channels
    hls = cv2.cvtColor(np.copy(image), cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    return l_channel, s_channel
    

def create_binary_images(image, s_channel_thresholds = (170, 255), x_derivative_thresholds = (20, 100)):        
    l_channel, s_channel = get_l_channel_and_s_channel(image)    
    x_derivative_binary = detect_edges_using_x_derivative(l_channel, x_derivative_thresholds)
    s_channel_binary = apply_thresholds(s_channel, s_channel_thresholds)
    
    return (create_color_binary_image(green_image = x_derivative_binary, blue_image = s_channel_binary),
            combine_with_or(x_derivative_binary, s_channel_binary))

In [ ]:
image = mpimg.imread('test_images/test2.jpg')
color_binary, combined_binary = create_binary_images(image)

# Plot the result
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(48, 18))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image')

ax2.imshow(color_binary)
ax2.set_title('color_binary')

ax3.imshow(combined_binary, cmap = 'gray')
ax3.set_title('combined_binary')

savefig('output_images/test1_binary.png')

## Apply a perspective transform to rectify binary image ("birds-eye view")

see https://classroom.udacity.com/nanodegrees/nd013/parts/edf28735-efc1-4b99-8fbb-ba9c432239c8/modules/5d1efbaa-27d0-4ad5-a67a-48729ccebd9c/lessons/78afdfc4-f0fa-4505-b890-5d8e6319e15c/concepts/ae58b4d0-b909-4f4b-9332-67d80a1b4029

In [ ]:
def get_img_src_dst():
    img = mpimg.imread('test_images/straight_lines1.jpg')
    src = np.float32(
        [
            ((get_width(img) / 2) - 55, get_height(img) / 2 + 100),
            (((get_width(img) / 6) - 10), get_height(img)),
            ((get_width(img) * 5 / 6) + 60, get_height(img)),
            ((get_width(img) / 2 + 55), get_height(img) / 2 + 100)
        ])
    dst = np.float32(
        [
            ((get_width(img) / 4), 0),
            ((get_width(img) / 4), get_height(img)),
            ((get_width(img) * 3 / 4), get_height(img)),
            ((get_width(img) * 3 / 4), 0)
        ])
    return img, src, dst
    
def getPerspectiveTransform():
    _, src, dst = get_img_src_dst()
    return cv2.getPerspectiveTransform(src, dst)

def warpPerspective(image):
    M = getPerspectiveTransform()
    warped = cv2.warpPerspective(image, M, get_imageSize_for_cv2(image), flags=cv2.INTER_NEAREST)
    return warped

In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Read in an image
img, src, dst = get_img_src_dst()
print("src:\n", src)
print("dst:\n", dst)

undist = undistort(img, get_calibration_data())

def polylines(pts, image):
    cv2.polylines(image, [pts.astype(int).reshape((-1, 1, 2))], True, (255, 0, 0), 6)

undist_with_source_points = np.copy(undist)
polylines(src, undist_with_source_points)

warped = warpPerspective(undist)
polylines(dst, warped)

f, (ax1, ax2) = plt.subplots(1, 2, figsize = (24, 9))
f.tight_layout()
ax1.imshow(undist_with_source_points)
ax1.set_title('Undistorted image with source points drawn', fontsize = 25)
ax2.imshow(warped)
ax2.set_title('Warped result with destination points drawn', fontsize = 25)
plt.subplots_adjust(left = 0., right = 1, top = 0.9, bottom = 0.)

savefig('output_images/test2_warped_straight_lines.png')

## Detect lane pixels and fit to find the lane boundary

https://classroom.udacity.com/nanodegrees/nd013/parts/edf28735-efc1-4b99-8fbb-ba9c432239c8/modules/5d1efbaa-27d0-4ad5-a67a-48729ccebd9c/lessons/626f183c-593e-41d7-a828-eda3c6122573/concepts/4dd9f2c2-1722-412f-9a02-eec3de0c2207

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[get_height(binary_warped)//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(get_height(binary_warped)//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = get_height(binary_warped) - (window+1)*window_height
        win_y_high = get_height(binary_warped) - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,
                      (win_xleft_low, win_y_low),
                      (win_xleft_high, win_y_high),
                      (0, 255, 0),
                      2) 
        cv2.rectangle(out_img,
                      (win_xright_low, win_y_low),
                      (win_xright_high, win_y_high),
                      (0, 255, 0),
                      2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) &
                          (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) &
                          (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) &
                           (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) &
                           (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def polyfit_line(y, x, pix2Meter):
    coeffs = np.polyfit(pix2Meter.y_pix_2_meter(y), pix2Meter.x_pix_2_meter(x), 2)
    return np.poly1d(coeffs)


def polyfit_left_line_right_line(lane_pixels, height, pix2Meter):
    leftx, lefty, rightx, righty, _ = lane_pixels
    left_line = polyfit_line(lefty, leftx, pix2Meter)
    right_line = polyfit_line(righty, rightx, pix2Meter)
    y = pix2Meter.y_pix_2_meter(np.linspace(0, height-1, height))
    return left_line, right_line, y


def draw_line(x, y, image):
    pts = np.dstack((x.astype(int), y.astype(int)))[0]
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], False, (255, 255, 0), 3)

    
def fit_polynomial(height, lane_pixels):
    leftx, lefty, rightx, righty, out_img = lane_pixels
    left_line, right_line, y = polyfit_left_line_right_line(lane_pixels = lane_pixels,
                                                            height = height,
                                                            pix2Meter = Pix2Meter(ym_per_pix = 1, xm_per_pix = 1))

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    # Plots the left and right polynomials on the lane lines
    draw_line(left_line(y), y, out_img)
    draw_line(right_line(y), y, out_img)
    return out_img, left_line(y), right_line(y), y

## Determine the curvature of the lane and vehicle position with respect to center


see https://classroom.udacity.com/nanodegrees/nd013/parts/edf28735-efc1-4b99-8fbb-ba9c432239c8/modules/5d1efbaa-27d0-4ad5-a67a-48729ccebd9c/lessons/626f183c-593e-41d7-a828-eda3c6122573/concepts/1a352727-390e-469d-87ea-c91cd78869d6

In [ ]:
def R_curve(line, y):
    return ((1 + (2*line.coef[0]*y + line.coef[1])**2)**1.5) / np.absolute(2*line.coef[0])

def measure_curvature_real(image, lane_pixels):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    left_line, right_line, y = polyfit_left_line_right_line(lane_pixels = lane_pixels,
                                                            height = get_height(image),
                                                            pix2Meter = create_default_pix2Meter())

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(y)
    
    # Calculation of R_curve (radius of curvature)
    return R_curve(left_line, y_eval), R_curve(right_line, y_eval)

def get_curvature(image, lane_pixels):
    left_curverad, right_curverad = measure_curvature_real(warped, lane_pixels)
    return (left_curverad + right_curverad)/2

For the position of the vehicle, you may assume the camera is mounted at the center of the car and the deviation of the midpoint of the lane from the center of the image is the offset you're looking for. As with the polynomial fitting, convert from pixels to meters. 

In [ ]:
def get_vehicle_position(lane_pixels, image, x_pix_2_meter):
    left_line, right_line, y = polyfit_left_line_right_line(lane_pixels = lane_pixels,
                                                            height = get_height(image),
                                                            pix2Meter = create_default_pix2Meter())

    def get_midpoint_of_lane():
        y_eval = np.max(y)
        return (left_line(y_eval) + right_line(y_eval))/2

    def get_center_of_car():
        return x_pix_2_meter(get_width(image)/2)

    return get_center_of_car() - get_midpoint_of_lane()

## Warp the detected lane boundaries back onto the original image

see https://classroom.udacity.com/nanodegrees/nd013/parts/edf28735-efc1-4b99-8fbb-ba9c432239c8/modules/5d1efbaa-27d0-4ad5-a67a-48729ccebd9c/lessons/7cb63828-36aa-4cea-9239-700b5ea41f0b/concepts/7ee45090-7366-424b-885b-e5d38210958f

In [ ]:
# Create an image to draw the lines on
def project_lane_area_onto_undistorted_image(warped, undist, Minv, lane_pixels):
    out_img, left_fitx, right_fitx, ploty = fit_polynomial(height = get_height(warped), lane_pixels = lane_pixels)

    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, get_imageSize_for_cv2(image))
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result, out_img

## Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

In [ ]:
def putText(image, text, bottomLeftCornerOfText):
    cv2.putText(img = image,
                text = text,
                org = bottomLeftCornerOfText,
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1.5,
                color = (255, 255, 255),
                thickness = 2)

# TODO: vielleicht den Durchschnitt von left_curverad und right_curverad ausgeben? Ist das mathematisch sinnvoll?
def printInfos(image, curverad, vehicle_position):
    radius_of_curvature_text = "Radius of Curvature = {:.2f} km".format(curverad/1000)
    putText(image, radius_of_curvature_text, (50, 50))
    
    vehicle_position_text = "Vehicle is {:.0f} cm {} of center".format(
        abs(vehicle_position*100),
        'right' if vehicle_position>0 else 'left')
    putText(image, vehicle_position_text, (50, 100))

## Run Pipeline

In [ ]:
from numpy.linalg import inv
from enum import Enum

class Image(Enum):
    INPUT = 'INPUT'
    UNDISTORTED = 'UNDISTORTED'
    COLOR_BINARY = 'COLOR_BINARY'
    COMBINED_BINARY = 'COMBINED_BINARY'
    PERSPECTIVE = 'PERSPECTIVE'
    LINES_WITH_SLIDING_WINDOWS = 'LINES_WITH_SLIDING_WINDOWS'
    LINES = 'LINES'
    OUTPUT = 'OUTPUT'

def pipeline(image):
    results = {Image.INPUT: image}
    calibration_data = get_calibration_data()
    
    image = undist = undistort(image, calibration_data)
    results[Image.UNDISTORTED] = undist
    
    color_binary, image = create_binary_images(image)
    results[Image.COLOR_BINARY] = color_binary
    results[Image.COMBINED_BINARY] = image
    
    image = warped = warpPerspective(image)
    results[Image.PERSPECTIVE] = warped
    
    lane_pixels = find_lane_pixels(warped)
    
    image, out_img = project_lane_area_onto_undistorted_image(warped,
                                                              undist,
                                                              inv(getPerspectiveTransform()),
                                                              lane_pixels)
    results[Image.LINES_WITH_SLIDING_WINDOWS] = out_img
    results[Image.LINES] = np.copy(image)
    
    printInfos(image,
               curverad = get_curvature(warped, lane_pixels),
               vehicle_position = get_vehicle_position(lane_pixels,
                                                       warped,
                                                       create_default_pix2Meter().x_pix_2_meter))
    results[Image.OUTPUT] = image
    return results

In [ ]:
def run_pipeline_and_draw(image):
    image = mpimg.imread(image)
    # image = cv2.imread('test_images/straight_lines2.jpg')
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    result = pipeline(image)[Image.OUTPUT]

    f.tight_layout()

    ax1.imshow(image)
    ax1.set_title('image -> pipeline', fontsize = 50)

    ax2.imshow(result, cmap = 'gray')
    ax2.set_title('pipeline -> image', fontsize = 50)

    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def run_pipeline_draw_save(input_image, ouput_image_pattern):
    image = mpimg.imread(input_image)
    results = pipeline(image)
    for image in results:
        plt.figure(figsize=(10,10))
        plt.imshow(results[image], cmap = 'gray')
        savefig(ouput_image_pattern.format(image))

In [ ]:
run_pipeline_draw_save(input_image = 'test_images/test2.jpg', ouput_image_pattern = 'output_images/test2_{}.png')

In [ ]:
run_pipeline_and_draw('test_images/straight_lines1.jpg')
run_pipeline_and_draw('test_images/straight_lines2.jpg')
run_pipeline_and_draw('test_images/test1.jpg')
run_pipeline_and_draw('test_images/test2.jpg')
run_pipeline_and_draw('test_images/test3.jpg')
run_pipeline_and_draw('test_images/test4.jpg')
run_pipeline_and_draw('test_images/test5.jpg')
run_pipeline_and_draw('test_images/test6.jpg')

## Video

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
process_image = pipeline

In [ ]:
def process_video(input_video, output_video):
    clip = VideoFileClip(input_video)
    clip_processed = clip.fl_image(process_image)
    %time clip_processed.write_videofile(output_video, audio=False)

In [ ]:
def embed_video(video):
    return HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(video))

In [ ]:
output_video = 'test_videos_output/project_video.mp4'
#process_video(input_video = 'test_videos/project_video.mp4', output_video = output_video)
embed_video(output_video)

In [ ]:
output_video = 'test_videos_output/challenge_video.mp4'
#process_video(input_video = 'test_videos/challenge_video.mp4', output_video = output_video)
embed_video(output_video)

In [ ]:
output_video = 'test_videos_output/harder_challenge_video.mp4'
#process_video(input_video = 'test_videos/harder_challenge_video.mp4', output_video = output_video)
embed_video(output_video)